In [2]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import playerdashboardbyteamperformance
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamdashboardbyteamperformance
from nba_api.stats.endpoints import playerdashboardbylastngames
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.endpoints import teamdashboardbylastngames
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas as pd
import inspect
import numpy as np
import csv
import sys

In [26]:
#Player Input

player1_input = input('Enter player 1: ')
print(player1_input)

player1 = players.find_players_by_full_name(player1_input)[0]['id']
player1_name = players.find_players_by_full_name(player1_input)[0]['full_name']

player_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=player1, season='2018-19')

# player_headers = player_info.by_year_player_dashboard.get_dict().get('headers')
# player_data = player_info.overall_player_dashboard.get_dict().get('data')[0]

# #last 20 games
player_headers = player_info.last20_player_dashboard.get_dict().get('headers')
player_data = player_info.last20_player_dashboard.get_dict().get('data')[0]

print(player_headers[5], player_data[5], 
	player_headers[2], player_data[2], 
	player_headers[26], player_data[26],
	player_headers[27], player_data[27])

Enter player 1: lebron james
lebron james
W_PCT 0.35 GP 20 PTS 553 PLUS_MINUS 19


In [68]:
#Team Input

team1_input = input('Enter team 1: ')
print(team1_input)
team1 = teams.find_teams_by_full_name(team1_input)[0]['id']

teamdashboard = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2018-19', last_n_games=1)
teamdashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2018-19', 
                                                                    measure_type_detailed_defense='Advanced', 
                                                                        last_n_games=1)

##last n games
team_headers_basic = teamdashboard.overall_team_dashboard.get_dict().get('headers')
team_data_basic = teamdashboard.overall_team_dashboard.get_dict().get('data')[0]
team_headers = teamdashboard_adv.overall_team_dashboard.get_dict().get('headers')
team_data = teamdashboard_adv.overall_team_dashboard.get_dict().get('data')[0]

##last 20 games
# team_headers_basic = teamdashboard.last20_team_dashboard.get_dict().get('headers')
# team_data_basic = teamdashboard.last20_team_dashboard.get_dict().get('data')[0]
# team_headers = teamdashboard_adv.last20_team_dashboard.get_dict().get('headers')
# team_data = teamdashboard_adv.last20_team_dashboard.get_dict().get('data')[0]

print(team1, team_headers, team_data)
# print(team1, team_headers[23], team_data[23], team_headers[8], team_data[8], team_headers[10], team_data[10], 
#       team_headers[2], team_data[2], team_headers[3], team_data[3], 
#       team_headers[4], team_data[4], team_headers[21], team_data[21], team_headers[6], team_data[6])

Enter team 1: lakers
lakers


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [57]:
roster1 = teamplayerdashboard.TeamPlayerDashboard(team_id=team1, last_n_games=1, season='2018-19').get_dict()#.PlayersSeasonTotals(team_id=team1, last_n_games=20)
team1_roster_last20 = []
team1_roster_lastn = []
#print(roster1)['resultSets'][1])#['rowSet'])
for n in roster1['resultSets'][1]['rowSet']:
    print(n[2], 'GP:', n[3], 'id:', n[1], 'Min', n[7])
    team1_roster_lastn.append(n[2])
    
print(team1_roster_lastn)
#(PER * MP)/n_players

Andrew Wiggins GP: 1 id: 203952 Min 35.983333333333334
Anthony Tolliver GP: 1 id: 201229 Min 21.083333333333332
C.J. Williams GP: 1 id: 203710 Min 17.6
Cameron Reynolds GP: 1 id: 1629244 Min 25.95
Dario Saric GP: 1 id: 203967 Min 29.255
Gorgui Dieng GP: 1 id: 203476 Min 34.445
Jared Terrell GP: 1 id: 1629123 Min 5.716666666666667
Josh Okogie GP: 1 id: 1629006 Min 17.116666666666667
Keita Bates-Diop GP: 1 id: 1628966 Min 18.583333333333332
Tyus Jones GP: 1 id: 1626145 Min 34.266666666666666
['Andrew Wiggins', 'Anthony Tolliver', 'C.J. Williams', 'Cameron Reynolds', 'Dario Saric', 'Gorgui Dieng', 'Jared Terrell', 'Josh Okogie', 'Keita Bates-Diop', 'Tyus Jones']


In [114]:
def off_per_calculator(player, team):
    
    agg_PER = 0 
    agg_adj_PER = 0
    num_players = 0
    PER_MP = 0
    agg_min = 0
    
    for n in player:
        p_id = players.find_players_by_full_name(n)[0]['id']
        p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=p_id, season='2018-19',
                                                                        last_n_games=1)
        
        ##last 20 games        
#         player_headers = p_info.last20_player_dashboard.get_dict().get('headers')
#         player_data = p_info.last20_player_dashboard.get_dict().get('data')[0]
        
        player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
        player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]

 
    #####PER player stats####
        MP = player_data[6]
        ThrP = player_data[10] 
        AST = player_data[19]
        TOV = player_data[20]
        FGA = player_data[8]
        FG = player_data[7] + player_data[10] 
        FTA = player_data[14]
        FT = player_data[13]
        TRB = player_data[18]
        ORB = player_data[16]
        STL = player_data[21]
        BLK = player_data[22]
        PF = player_data[24]

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = team_data_basic[19]
        team_FG = team_data_basic[7]+team_data_basic[10]
        team_pace = team_data[23] 
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)
        
        agg_PER += PER_calc_player 
        agg_adj_PER += adj_PER_calc_player
        num_players +=1
        PER_MP += (adj_PER_calc_player * MP)
        PER_MP_n = PER_MP/num_players
        
        print(n, 'Min', MP, 'adj_PER_calc_player', adj_PER_calc_player, 'PER*MP_Player', (adj_PER_calc_player*MP))
        
    print('Agg_PER * MP', PER_MP)
    
    print('PERxMP/n', PER_MP_n, 'Total PER', agg_PER, 'Total adj_PER', agg_adj_PER)
    print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)
#    print('league stats:', factor, VOP, DRB_perc)

In [115]:
off_per_calculator(team1_roster_lastn, team1)

Danilo Gallinari Min 24.8 adj_PER_calc_player 0.426271056923702 PER*MP_Player 10.57152221170781
Garrett Temple Min 21.583333333333332 adj_PER_calc_player 0.3514380730823768 PER*MP_Player 7.585205077361299
Ivica Zubac Min 29.783333333333335 adj_PER_calc_player 0.6585686980390959 PER*MP_Player 19.61437105659774
JaMychal Green Min 25.333333333333332 adj_PER_calc_player 0.5061284203324381 PER*MP_Player 12.821919981755098
Jerome Robinson Min 16.45 adj_PER_calc_player 0.22087037238040508 PER*MP_Player 3.6333176256576634
Landry Shamet Min 29.416666666666668 adj_PER_calc_player 0.5033830669166712 PER*MP_Player 14.807851885132079
Lou Williams Min 23.316666666666666 adj_PER_calc_player 0.3062138270859272 PER*MP_Player 7.139885734886869
Montrezl Harrell Min 20.816666666666666 adj_PER_calc_player 0.9940717283930562 PER*MP_Player 20.693259812715453
Patrick Beverley Min 21.316666666666666 adj_PER_calc_player 1.013862675943175 PER*MP_Player 21.612172708855347
Shai Gilgeous-Alexander Min 24.1 adj_PER_

In [136]:
#Alternate Offensive Rating

def off_per_calc_alt(player, team):
    
    agg_off_rating = 0
    agg_min = 0

    player_info_off = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=5, season='2019-20', 
                                                                  measure_type_detailed_defense='Advanced')

    player_off_headers = player_info_off.league_dash_player_stats.get_dict().get('headers')
    player_off_data = player_info_off.league_dash_player_stats.get_dict().get('data')

#     print('headers:', player_off_headers)
    #print('data:', player_def_data)

    for p in team1_roster_lastn:
        for n in player_off_data:
            if p in n:
                print(n[1], '\n', 'E_Off_Rating', n[10], 'Min', n[9])
                agg_off_rating += (n[10]*n[9])
                agg_min += n[9]

    avg_off_rating = agg_off_rating/len(team1_roster_lastn)

    print('agg_off_rating', agg_off_rating, 'avg_off_rating', avg_off_rating, 'agg_min', agg_min, 
          'agg_off_rat_min', agg_off_rating/agg_min) 

In [107]:
#Individual Player Defense Stats

def def_per_calc(team_roster):
    agg_def_rating = 0
    agg_min = 0

    # player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=1, season='2018-19', 
    #                                                                   measure_type_detailed_defense='Defense')
    player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(opponent_team_id=, month=4, season='2018-19', 
                                                                      measure_type_detailed_defense='Defense')



    player_def_headers = player_info_def.league_dash_player_stats.get_dict().get('headers')
    player_def_data = player_info_def.league_dash_player_stats.get_dict().get('data')

    print('headers:', player_def_headers)
        #print('data:', player_def_data)

    for p in team1_roster_lastn:
        for n in player_def_data:
            if p in n:
                print(n[1], '\n', 'Def Rating', n[10], 'Min', n[9])
                agg_def_rating += (n[10]*n[9])
                agg_min += n[9]


    avg_def_rating = agg_def_rating/len(team1_roster)
    avg_def_rating_min = agg_def_rating/agg_min

    print('agg_def_rating', agg_def_rating, 'avg_def_rating', avg_def_rating, 'agg_min', agg_min) 
    print('avg_def_rating_min', avg_def_rating_min)

In [66]:
def off_per_calculator_pandas(player, team):
    
    agg_PER = 0 
    agg_adj_PER = 0
    num_players = 0
    PER_MP = 0
    agg_min = 0

    
    for n in player:
#         p_id = players.find_players_by_full_name(n)[0]['id']
#         p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=p_id, season='2018-19',
#                                                                         last_n_games=1)
        
#         ##last 20 games        
# #         player_headers = p_info.last20_player_dashboard.get_dict().get('headers')
# #         player_data = p_info.last20_player_dashboard.get_dict().get('data')[0]
        
#         player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
#         player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]

 
    #####PER player stats####
        MP = team1_roster['MINUTES']
        ThrP = team1_roster['3PMa'] 
        AST = team1_roster['AST']
        TOV = team1_roster['TOV']
        FGA = team1_roster['FGA']
        FG = team1_roster['FGM'] 
        FTA = team1_roster['FTA']
        FT = team1_roster['FTM']
        TRB = team1_roster['REB']
        ORB = team1_roster['OREB']
        STL = team1_roster['STL']
        BLK = team1_roster['BLK']
        PF = team1_roster['PF']

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = 25.6
        team_FG = 42.6
        team_pace = 103.6
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)
        
        agg_PER += PER_calc_player 
        agg_adj_PER += adj_PER_calc_player
        num_players +=1
        PER_MP = adj_PER_calc_player * MP
        PER_MP_n += PER_MP
        
        print(n, 'Min', MP, 'adj_PER_calc_player', adj_PER_calc_player, 'PER*MP_Player', (adj_PER_calc_player*MP))
        
    print('Agg_PER * MP_n', PER_MP)
    
#     print('PERxMP/n', PER_MP_n, 'Total PER', agg_PER, 'Total adj_PER', agg_adj_PER)
    print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)
#    print('league stats:', factor, VOP, DRB_perc)

In [90]:
data = pd.read_csv("NBA Merged Sheets - Sheet2.csv") 
df = pd.read_csv("team 2018-19.csv") 

team1 = 'ATL'

df = df[['Team', 'Matchup', 'Date']]
df = df[df['Team'] == team1]

df[df['Team']==team1].any()
print(df)



# team1_data = data[data.TEAM == team1]

# team1_data['game_num'] = list(zip(team1_data.TEAM, team1_data.DATE))
# team1_data['game_num'] = pd.factorize(team1_data['game_num'])[0]

# team1_roster = []
# opponent = []
# count = 0
# team1_roster_players = []

# pd.set_option('display.max_columns', None)

# while count < 1:
#     for p in team1_data:
#         team1_roster = team1_data[team1_data['game_num']==count]
#     team1_roster_players.append(team1_roster['PLAYER'])
    
#     count+=1

    
    
    
    
# off_per_calculator_pandas(team1_roster_players, team1)
    
    
    
# # def_per_calc(team1_roster)



Empty DataFrame
Columns: [Team, Matchup, Date]
Index: []


In [ ]:
#Theory: If second game of back-to-back is a road game, odds of winning decrease 7-8%, multiply ratings by 0.93.